In [1]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
from glob import glob
import cv2
import numpy as np

In [61]:
# Load labeled frames and their corresponding labels
def load_labeled_frames(data_folder):
    images = []
    labels = []

    for image_path in glob(os.path.join(data_folder, '*.jpg')):
        image = cv2.imread(image_path)
        image = cv2.resize(image, (50, 50))

        label = os.path.splitext(os.path.basename(image_path))[0].split('_')[-1]

        images.append(image)

        if label == '0':
            labels.append(0)
        elif label == '1':
            labels.append(1)

    return images, labels

# Preprocess the data
def preprocess_data(images, labels):
    images = np.array(images)
    labels = np.array(labels)
    
    # duplicate the images and labels
    images = np.concatenate((images, images), axis=0)
    labels = np.concatenate((labels, labels), axis=0)

    # Normalize pixel values to be between 0 and 1
    images = images / 255.0

    # Shuffle and split the data into training and testing sets
    images, labels = shuffle(images, labels, random_state=46)
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=45)

    return X_train, X_test, y_train, y_test

In [62]:
# Example usage
output_folder = 'labeled_frames_binary_test'

# Load labeled frames and their corresponding labels
images, labels = load_labeled_frames(output_folder)

# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_data(images, labels)

input_shape = X_train[0].shape

In [63]:
y_train

array([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0])

In [64]:
# Build the model
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid')) 

# Compile the model
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',  # Use binary crossentropy for binary classification
              metrics=['accuracy', 'AUC'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))


Epoch 1/30
3/3 [==============================] - 9s 922ms/step - loss: 0.6790 - accuracy: 0.5349 - auc: 0.4426 - val_loss: 0.6002 - val_accuracy: 0.6364 - val_auc: 0.9643
Epoch 2/30
3/3 [==============================] - 1s 310ms/step - loss: 0.6001 - accuracy: 0.6512 - auc: 0.6923 - val_loss: 0.5702 - val_accuracy: 0.6364 - val_auc: 1.0000
Epoch 3/30
3/3 [==============================] - 1s 271ms/step - loss: 0.5717 - accuracy: 0.6512 - auc: 0.7399 - val_loss: 0.5603 - val_accuracy: 0.6364 - val_auc: 1.0000
Epoch 4/30
3/3 [==============================] - 1s 339ms/step - loss: 0.4994 - accuracy: 0.7326 - auc: 0.9702 - val_loss: 0.5724 - val_accuracy: 0.6364 - val_auc: 0.8571
Epoch 5/30
3/3 [==============================] - 1s 308ms/step - loss: 0.4557 - accuracy: 0.7326 - auc: 0.9640 - val_loss: 0.5535 - val_accuracy: 0.6364 - val_auc: 0.9286
Epoch 6/30
3/3 [==============================] - 1s 256ms/step - loss: 0.4081 - accuracy: 0.8023 - auc: 0.9604 - val_loss: 0.5761 - val_acc

In [65]:
# Evaluate the model
test_loss, test_acc, test_auc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest AUC:', test_auc)

1/1 - 0s - loss: 0.0128 - accuracy: 1.0000 - auc: 1.0000 - 87ms/epoch - 87ms/step

Test AUC: 1.0


In [67]:
import cv2
import numpy as np

# Open the video file
video_path = './vidscapstone/vid9.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

rois = [(489, 809, 53, 52), (1388, 265, 53, 54), (443, 268, 57, 57), (1359, 758, 57, 57)]

# Loop through frames
while True:
    print('________________________________________________________')
    # Read a frame
    ret, frame = cap.read()

    # Break the loop if the video is finished
    if not ret:
        break

    count = 0

    for roi in rois:
        x, y, w, h = roi
        roi_processed = frame[y:y+h, x:x+w]
        roi_processed = cv2.resize(roi_processed, (50, 50))
        roi_processed = cv2.convertScaleAbs(roi_processed)
        roi_processed = np.array(roi_processed)
        roi_processed = roi_processed / 255.0
        roi_processed = np.expand_dims(roi_processed, axis=0)
        pred = model.predict(roi_processed)
        print(pred)
        if pred > 0.5:
            count += 1

        cv2.imshow(f'ROI {x}, {y}', frame[y:y+h, x:x+w])

        breaker = False
        # Press q to exit
        # if cv2.waitKey(0) & 0xFF == ord('q'):
        #     cv2.destroyAllWindows()
        #     breaker = True
        #     break
        # else:
        #     cv2.destroyAllWindows()

    if breaker:
        break


    #frame_to_show = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    #cv2.imshow(f'Turned on lights {count}', frame_to_show)
    
    # print(f'Turned on lights: {count}')

# Release the video capture object
cap.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


________________________________________________________
1/1 [==============================] - 0s 64ms/step
[[0.01731532]]
1/1 [==============================] - 0s 243ms/step
[[1.]]
1/1 [==============================] - 0s 119ms/step
[[1.]]
1/1 [==============================] - 0s 76ms/step
[[1.]]
________________________________________________________
1/1 [==============================] - 0s 76ms/step
[[1.]]
1/1 [==============================] - 0s 70ms/step
[[1.]]
1/1 [==============================] - 0s 77ms/step
[[1.]]
1/1 [==============================] - 0s 62ms/step
[[1.]]
________________________________________________________
1/1 [==============================] - 0s 58ms/step
[[1.]]
1/1 [==============================] - 0s 78ms/step
[[0.01731532]]
1/1 [==============================] - 0s 95ms/step
[[1.]]
1/1 [==============================] - 0s 73ms/step
[[1.]]
________________________________________________________
1/1 [==============================] - 0s 85ms

In [ ]:
import pickle

directory = 'testing_binary_class_classification/'

model.save(directory + 'testing_binary_class_classification_model_Adam.h5')

with open(directory + 'testing_binary_class_classification_model_Adam_history', 'wb') as file_pi:
    pickle.dump(history, file_pi)

with open(directory + 'testing_binary_class_classification_model_Adam_X_test', 'wb') as file_pi:
    pickle.dump(X_test, file_pi)

with open(directory + 'testing_binary_class_classification_model_Adam_y_test', 'wb') as file_pi:
    pickle.dump(y_test, file_pi)